# Vraag 2 - Opdracht 1
 Bouw een systeem waar je anomalies detecteert aan de hand van: XGboost algoritme.

## Importeer benodigde bibliotheken

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.preprocessing import StandardScaler

## Laad en bescrhijf de dataset

In [ ]:
data = pd.read_csv('creditcard.csv')
data.describe()

NameError: ignored

Controleer missing values

In [ ]:
data.isnull().sum().sum()

0

Bekijk verschillende kolommen

In [ ]:
data.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

De enige niet vervormde variablen zijn:
- Time
- Amount
- Class (1: fraude & 0: niet fraude)

In [ ]:
vc = data['Class'].value_counts().to_frame().reset_index()
vc['percent'] = vc["Class"].apply(lambda x : round(100*float(x) / len(data), 2))
vc = vc.rename(columns = {"index" : "Target", "Class" : "Count"})
vc

0    284315
1       492
Name: Class, dtype: int64

## Data Voorbereiden

In [ ]:
scalar = StandardScaler()

X = data.drop('Class', axis=1)
y = data.Class

X_train_v, X_test, y_train_v, y_test = train_test_split(X, y,
                                                    test_size=0.3, random_state=42)
X_train, X_validate, y_train, y_validate = train_test_split(X_train_v, y_train_v,
                                                            test_size=0.2, random_state=42)

X_train = scalar.fit_transform(X_train)
X_validate = scalar.transform(X_validate)
X_test = scalar.transform(X_test)



Fraudulant transaction weight: 0.0017994745785028623
Non-Fraudulant transaction weight: 0.9982005254214972
TRAINING: X_train: (159491, 30), y_train: (159491,)
_______________________________________________________
VALIDATION: X_validate: (39873, 30), y_validate: (39873,)
__________________________________________________
TESTING: X_test: (85443, 30), y_test: (85443,)


# XGBoost model

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train, eval_metric='aucpr')

y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)




/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


Train Result:
Accuracy Score: 100.00%
_______________________________________________
Classification Report:
                  0      1  accuracy  macro avg  weighted avg
precision       1.0    1.0       1.0        1.0           1.0
recall          1.0    1.0       1.0        1.0           1.0
f1-score        1.0    1.0       1.0        1.0           1.0
support    159204.0  287.0       1.0   159491.0      159491.0
_______________________________________________
Confusion Matrix: 
 [[159204      0]
 [     0    287]]

Test Result:
Accuracy Score: 99.96%
_______________________________________________
Classification Report:
                      0           1  accuracy     macro avg  weighted avg
precision      0.999707    0.948718  0.999637      0.974212      0.999626
recall         0.999930    0.816176  0.999637      0.908053      0.999637
f1-score       0.999818    0.877470  0.999637      0.938644      0.999624
support    85307.000000  136.000000  0.999637  85443.000000  85443.000000


Evaluatie

In [ ]:
# Evaluatie
# Functie om evaluatiemetrieken af te drukken
def print_test_score(label, prediction):
  if train:
    print("Train Resultaat:\n================================================")
    print(f"Nauwkeurigheids Score: {accuracy_score(label, prediction) * 100:.2f}%")
    print("_______________________________________________")
    print(f"Classificatie Rapport:\n{classification_report(label, prediction)}")
    print("_______________________________________________")
    print(f"Confusion Matrix: \n {confusion_matrix(label, prediction)}\n")

    elif train==False:
      print("Train resultaat:\n================================================")
      print(f"Nauwkeurigheids Score: {accuracy_score(label, prediction) * 100:.2f}%")
      print("_______________________________________________")
      print(f"Classificatie Rapport:\n{classification_report(label, prediction)}")
      print("_______________________________________________")
      print(f"Confusion Matrix: \n {confusion_matrix(label, prediction)}\n")

print_score(y_train, y_train_pred, train=True)
print_score(y_test, y_test_pred, train=False)

---

## Confusion Matrix reports:
`[[TN, FP] `

**True Negatives (TN):** Dit zijn de gevallen waarin uw model correct voorspelt dat een transactie normaal is.

**False Positives (FP):** Dit zijn de gevallen waarin uw model onterecht voorspelt dat een normale transactie frauduleus is.


`[FN, TP]]`

**False Negatives (FN):** Dit zijn de frauduleuze transacties die uw model niet heeft gedetecteerd.


**True Positives (TP):** Dit zijn de gevallen waarin uw model correct voorspelt dat de transactie frauduleus is.

---

## Classification Report:
Het classificatierapport bevat verschillende belangrijke statistieken:

**Precision**: Geeft aan hoeveel van de door het model als positief (frauduleus) geclassificeerde gevallen daadwerkelijk positief zijn.

**Recall**: Geeft aan hoeveel van de werkelijke positieve gevallen het model correct heeft geïdentificeerd.

**F1-score**: De harmonische gemiddelde van precisie en recall, een balans tussen de twee.

**Support**: Het aantal werkelijke gevallen voor elke klasse in de dataset.

**Accuracy**: Het totale percentage correcte voorspellingen door het model.

**Macro avg**: Het ongewogen gemiddelde van de statistieken voor elke klasse.

**Weighted avg**: Het gewogen gemiddelde van de statistieken voor elke klasse, waarbij de gewichten de ondersteuning (aantal gevallen) voor elke klasse zijn.




